In [1]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import plotly.express as px
from sklearn.model_selection import cross_val_score
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

In [35]:
df = pd.read_csv("retouren.csv")
df["user_dob"] = pd.to_datetime(df.user_dob)
df["user_reg_date"] = pd.to_datetime(df.user_reg_date)
df["order_date"] = pd.to_datetime(df.order_date)
df["delivery_date"] = pd.to_datetime(df.delivery_date)
df = df.drop(df[df.item_price == 999].index)
df = df.drop(df[df.delivery_date < df.order_date].index)
df = df.drop(df[df.item_color == "?"].index)
df.loc[df.item_color == "brwon","item_color"] = "brown"
df.loc[df.item_color == "blau","item_color"] = "blue"
df.loc[df.item_color == "dark oliv","item_color"] = "dark olive"
df["age"] = df.order_date.dt.year - df.user_dob.dt.year -((df.order_date.dt.date)<(df.user_dob.dt.date))
df["acc_age"] = df.order_date - df.user_reg_date 
df["acc_age"] = df.acc_age.dt.days
df_mark = df.groupby(["item_id","item_size","item_color"]).agg(maxprice=("item_price","max")).reset_index()
df = pd.merge(df,df_mark)
df["markdown"] = 0
df.loc[df.item_price < df.maxprice,"markdown"] =   df.item_price / df.maxprice
df.drop(df[df.delivery_date.isna()].index, inplace=True)

In [36]:
df.loc[df.age > 100, "age"] = df.loc[df.age > 100, "age"] - 100

In [18]:
px.scatter(df.groupby("age")[["age"]].count())

In [31]:
bins=[0,10,20,30,40,50,60,70,80,90,100]
df = pd.cut(df.age,bins=bins)

In [34]:
df[df.isna()]

6        NaN
17       NaN
22       NaN
51       NaN
61       NaN
        ... 
73984    NaN
74004    NaN
74027    NaN
74031    NaN
74041    NaN
Name: age, Length: 5616, dtype: category
Categories (10, interval[int64, right]): [(0, 10] < (10, 20] < (20, 30] < (30, 40] ... (60, 70] < (70, 80] < (80, 90] < (90, 100]]

In [37]:
bins=[0,10,20,30,40,50,60,70,80,90,100]
df["age"] = pd.cut(df.age,bins=bins)


In [38]:
df = pd.get_dummies(df.age,dummy_na=True)

In [40]:
df.head(10)

,"(0.0, 10.0]","(10.0, 20.0]","(20.0, 30.0]","(30.0, 40.0]","(40.0, 50.0]","(50.0, 60.0]","(60.0, 70.0]","(70.0, 80.0]","(80.0, 90.0]","(90.0, 100.0]",NaN
0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,1,0,0,0,0,0
10,0,0,0,0,1,0,0,0,0,0,0
11,0,0,0,1,0,0,0,0,0,0,0
12,0,0,0,0,1,0,0,0,0,0,0


sachen mit maybe relevanz: item_id,item_size idk vielleicht auch anhand der größe in objekte unterteilen,item_color(hot encoding ?),brand_id

item_price;user_id,user_title(hot encoding);user_state

Sachen die wir dazu machen können: age; Markdown;not delivered